In [1]:
import numpy as np
from scipy.optimize import root
import prop
import math as m

G2 = 8.3
P1 = 0.2
P2 = 0.1
T11 = 95
T12 = 70
T21 = -30
T22 =  20


HOT = 'REFPROP::WATER'
COLD = 'REFPROP::AIR'

Q = G2*(prop.t_p(T22,P2, COLD)["H"]-prop.t_p(T21,P2, COLD)["H"])
G1 = Q/(prop.t_p(T11,P1, HOT)["H"]-prop.t_p(T12,P1, HOT)["H"])

Dvnes=0.01
DeltaTube=0.001
Dvnut=Dvnes-2*DeltaTube
step=2.5*Dvnes        
Hchan=0.25*Dvnes
Deltaplate = 0.0005
lambdaw = 20
etta=1
w1 = 2
w2 = 10
Z = 5

T1av = (T11+T12)/2
T2av = (T21+T22)/2

ro1 = prop.t_p(T1av,P1, HOT)['ro']
ro2 = prop.t_p(T2av,P2, COLD)['ro']
nu1 = prop.t_p(T1av,P1, HOT)['nu']
nu2 = prop.t_p(T2av,P2, COLD)['nu']
lambda1 = prop.t_p(T1av,P1, HOT)['lamda']
lambda2 = prop.t_p(T2av,P2, COLD)['lamda']
Pr1 = prop.t_p(T1av,P1, HOT)['Pr']
Pr2 = prop.t_p(T2av,P2, COLD)['Pr']

dTmax = T11 - T22
dTmin = T12 - T21
dT = (dTmax - dTmin) / (np.log(dTmax / dTmin))

Re1 = w1*Dvnut/nu1
Nu1 = 0.021*Re1**0.8*Pr1**0.43
def func(x):
    delta = 10**-5
    left = x
    right = 1/(2*np.log10((2.51/(Re1*(x)**(1/2)))+(delta/3.7)))**2
    return left-right
sol = root(func, 0.01)
f1=float(sol.x)
alpha1 = Nu1*lambda1/Dvnut
Re2 = w2*Dvnes/nu2
Nu2 = 0.195*Re2**0.65*Pr2**(-1/3)
f2 = 1.7*Re2**(-0.5)
alpha2 = Nu2*lambda2/Dvnes
KF = Q*1000/dT

F_vneshn = 4*(((2**0.5)*step/2)**2 - np.pi*(Dvnes**2)/4) + 2*Hchan* np.pi*Dvnes
F_vntr = np.pi*(Dvnes-2*DeltaTube)*2*(Hchan+Deltaplate)
F_trube = np.pi*Dvnes*2*(Hchan+Deltaplate)
koef1 = F_vntr/F_vneshn
koef2 = F_vneshn / F_trube

def func(x):
    Fvnes=x
    a1 = 1/KF
    a2 = 1/(etta*alpha2*Fvnes)
    a3 = ((1/2)*(np.log(Dvnes/Dvnut))*(Dvnes/(lambdaw*(Fvnes/koef2))))
    a4 = 1/(alpha1*Fvnes*koef1)
    return a1-a2-a3-a4
sol = root(func, 0.002)
x=sol.x
Fvnes=float(x)
Nsumm = m.ceil(Fvnes/(F_vneshn)/Z)

S2 = G2/(ro2*w2)
S1 = G1/(ro1*w1)


NH =  m.ceil(Nsumm*(np.pi*Dvnut**2)/(S1*4))
NW =  m.ceil(S2/(4*NH*Hchan*(step-(Dvnut+2*DeltaTube))))
NL =  m.ceil(4*S1/(NW*np.pi*Dvnut**2))
S2_y = 4*NW*NH*Hchan*(step-(Dvnut+2*DeltaTube))
S1_y = NW*NL*np.pi*Dvnut**2/4


w1_y = G1/(ro1*S1_y)
w2_y = G2/(ro2*S2_y)
Nsumm_y = NH*NW*NL
Fvnes_y =Nsumm_y*F_vneshn*Z

W=round(NW*step, 4)
H=round(NH*((Hchan+Deltaplate)*2), 4)
L=round(NL*step*Z, 4)

dP1_y = f1*Z*H/Dvnes*ro1*w1_y**2/2
dP2_y = f2*Z*L/Dvnut*ro2*w2_y**2/2
    
Ntube = NW*NL*Z
Nplate = NH*2*Z
Ltube = 2* (Hchan+Deltaplate)*NH
Wplate = NW*step
Lplate = NL*step

V = np.pi*(Dvnes**2-Dvnut**2)/4*Ltube*Ntube + Lplate*Wplate*Deltaplate*Nplate
ro_stal = 7832 # кг/м3
stoimost_metal = 300 # руб за кг

Massa =   ro_stal * V 
Massa_tube  = np.pi*(Dvnes**2-Dvnut**2)/4*Ltube*Ntube * ro_stal
Massa_plate = Lplate*Wplate*Deltaplate*Nplate * ro_stal

Stoimost_tube = stoimost_metal*Massa_tube
Stoimost_plate = stoimost_metal*Massa_plate
Stoimost_TO = Stoimost_tube +Stoimost_plate 
Svarka = 75*Nplate
Korpys = 300000*V
money = round(Stoimost_tube + Stoimost_plate + Svarka + Korpys)

print('Длина ТО:', L)
print('Ширина ТО:', W)
print('Высота ТО:', H)
print('Площадь ТО:', Fvnes_y)
print('Потери давления по горячему каналу:',dP1_y)
print('Потери давления по холодному каналу:',dP2_y)
print('Стоимость ТО:',money)

Длина ТО: 0.25
Ширина ТО: 0.75
Высота ТО: 0.858
Площадь ТО: 46.88628375804991
Потери давления по горячему каналу: 8222.206612848171
Потери давления по холодному каналу: 194.16231969632716
Стоимость ТО: 197576
